In [1]:
from joblib import load

# Load the model for Zoom
kmeans = load('KMeans_NodeClustering.joblib')

/Users/hemanthsaiyeddulapalli/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KMeans from version 1.2.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
#Test on actual data
import pandas as pd

# Define the data as a list of dictionaries

data = [
    {'Node': 'VEC-Master', 'RAM (GB)': 4, 'Memory (GB)': 20, 'CPU (Cores)': 2, 'Type': 'Compute', 'Security Score (0-10)': 8},
    {'Node': 'VEC_VEN1', 'RAM (GB)': 0.5, 'Memory (GB)': 8, 'CPU (Cores)': 1, 'Type': 'Storage', 'Security Score (0-10)': 8},
    {'Node': 'VEC_VEN2', 'RAM (GB)': 1, 'Memory (GB)': 12, 'CPU (Cores)': 1, 'Type': 'Compute', 'Security Score (0-10)': 8},
    {'Node': 'VEC_VEN3', 'RAM (GB)': 2, 'Memory (GB)': 16, 'CPU (Cores)': 1, 'Type': 'Storage', 'Security Score (0-10)': 8},
    {'Node': 'VEC_VEN4', 'RAM (GB)': 4, 'Memory (GB)': 12, 'CPU (Cores)': 2, 'Type': 'Compute', 'Security Score (0-10)': 8},
    {'Node': 'VEC_VEN5', 'RAM (GB)': 4, 'Memory (GB)': 30, 'CPU (Cores)': 2, 'Type': 'Compute', 'Security Score (0-10)': 8}
]

'''
file_path = 'DummyData.csv'
# Create a DataFrame
#df = pd.DataFrame(file_path)
df = pd.read_csv(file_path)

# Now you have the dummy data as a pandas DataFrame.
# You can view the DataFrame by printing it.
df

# If you need to save this data to a CSV file, you can do so using the to_csv method.
#df.to_csv('dummy_node_data.csv', index=False)
'''

"\nfile_path = 'DummyData.csv'\n# Create a DataFrame\n#df = pd.DataFrame(file_path)\ndf = pd.read_csv(file_path)\n\n# Now you have the dummy data as a pandas DataFrame.\n# You can view the DataFrame by printing it.\ndf\n\n# If you need to save this data to a CSV file, you can do so using the to_csv method.\n#df.to_csv('dummy_node_data.csv', index=False)\n"

In [3]:
df = pd.DataFrame(data)

In [4]:
# One-hot encode the 'Type' column
type_dummies = pd.get_dummies(df['Type'], prefix='Type')

# Drop the original 'Type' column and concatenate the one-hot encoded columns
df_encoded = pd.concat([df.drop(['Type','Node'], axis=1), type_dummies], axis=1)

df_encoded

,RAM (GB),Memory (GB),CPU (Cores),Security Score (0-10),Type_Compute,Type_Storage
0,4.0,20,2,8,1,0
1,0.5,8,1,8,0,1
2,1.0,12,1,8,1,0
3,2.0,16,1,8,0,1
4,4.0,12,2,8,1,0
5,4.0,30,2,8,1,0


In [5]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

X = df_encoded.values
# Standardize the data to have a mean of ~0 and a variance of 1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create a KMeans instance with 3 clusters: this number can be changed according to your needs or determined by methods like the elbow method
#kmeans = KMeans(n_clusters=4) #got number of clusters from Elbow method 

# Fit model to data
kmeans.fit(X_scaled)

# Determine the cluster labels of each point
labels = kmeans.predict(X_scaled)

# Add the cluster labels as a column to the original DataFrame
#df['Cluster'] = labels
df_encoded['Cluster'] = labels

TypeError: '<=' not supported between instances of 'str' and 'int'

In [8]:
df_encoded

,RAM (GB),Memory (GB),CPU (Cores),Security Score (0-10),Type_Compute,Type_Storage,Cluster
0,4.0,20,2,8,1,0,3
1,0.5,8,1,8,0,1,1
2,1.0,12,1,8,1,0,2
3,2.0,16,1,8,0,1,1
4,4.0,12,2,8,1,0,3
5,4.0,30,2,8,1,0,0


In [9]:
df_new = df['Node']
df_new = pd.concat([df_new, df_encoded['Cluster']], axis=1)
df_new

,Node,Cluster
0,VEC-Master,3
1,VEC_VEN1,1
2,VEC_VEN2,2
3,VEC_VEN3,1
4,VEC_VEN4,3
5,VEC_VEN5,0


In [18]:
# Convert specific rows to a dictionary
Node_Cluster = df_new.set_index('Node')['Cluster'].to_dict()

Node_Cluster


{'VEC-Master': 3,
 'VEC_VEN1': 1,
 'VEC_VEN2': 2,
 'VEC_VEN3': 1,
 'VEC_VEN4': 3,
 'VEC_VEN5': 0}